In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unicodedata
from nltk.corpus import stopwords

In [70]:
pagamentos = pd.read_csv('pagamentos.csv')
pagamentos.info()
print(pagamentos.nunique())
pagamentos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427873 entries, 0 to 427872
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   data_publicacao          427873 non-null  object
 1   fase                     427873 non-null  object
 2   credor                   427873 non-null  object
 3   valor                    427873 non-null  object
 4   numero                   427873 non-null  object
 5   cpf_ou_cnpj              427873 non-null  object
 6   data_pagamento           427873 non-null  object
 7   numero_processo          253290 non-null  object
 8   bem_ou_servico_prestado  420978 non-null  object
 9   natureza                 422333 non-null  object
 10  acao                     407378 non-null  object
 11  funcao                   407414 non-null  object
 12  subfuncao                407405 non-null  object
 13  processo_licitatorio     407396 non-null  object
 14  fonte_recurso       

,data_publicacao,fase,credor,valor,numero,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
0,01/01/2010,EMPENHO,METALPIRES INDUSTRIA E COM LT,"R$ 203.690,00",10000348,06.965.538/0001-00,01/01/2010,157/2009,REF AQUISICAO DE MOBILIARIOS E EQUIPAMENTOS PA...,449052090000 - Out.Equip. e Mat. Permanente,2040 - Equipar e Reequipar Unidades Escolares ...,12 - EDUCACAO,361 - ENSINO FUNDAMENTAL,PREGAO,0001 - REC.IMP.TRANSF.EDUCACAO 25%
1,01/01/2010,EMPENHO,INST.NAC.DE SEGURIDADE SOCIAL,"R$ 1.500.000,00",10000195,01.404.357/0001-51,01/01/2010,NaN,REFERENTE A DESPESA COM PARCE- LAMENTO DA DI...,469071030000 - INSS/PARCELAMENTO,2145 - Administracao de encagos especiais,28 - ENCARGOS ESPECIAIS,843 - SERVICO DA DIVIDA INTERNA,ISENTO,0000 - RECURSOS ORDINARIOS
2,01/01/2010,EMPENHO,GASAUTO LOCAC?O DE VEICULOS L,"R$ 18.816,00",113/2009,00.426.288/0001-58,01/01/2010,039/2009,locac?o de 04 motocicletas 150cc com com parti...,339039780000 - locacao de veiculos,2141 - Seguranca no transito,26 - TRANSPORTE,452 - SERVICOS URBANOS,CONVITE,0050 - REC.PROPRIAS ENT.ADM.
3,02/01/2010,EMPENHO,SALUSTIANO JOSE ALVES DIAS,"R$ 4.800,00",10000024,407.981.865-34,02/01/2010,230/2007,"TERMO DE RENOVACAO DO ADITIVO 5025271/2009, R...",339039780000 - locacao de veiculos,2097 - Pavimentacao e infra-estrutura,15 - URBANISMO,451 - INFRA-ESTRUTURA URBANA,TOMADA DE PRECO,0000 - RECURSOS ORDINARIOS
4,02/01/2010,EMPENHO,VALDEANE DOS SANTOS DIAS,"R$ 3.600,00",10000494,006.534.185-66,02/01/2010,NaN,REF. ALUGUEL DO IMOVEL ONDE FUNCIONA O NUCL...,339039100000 - Locacao de imoveis,2084 - Programa Errad.Trabalho Infantil PETI,08 - ASSISTENCIA SOCIAL,243 - ASSISTENCIA A CRIANCA E AO ADO,ISENTO,0000 - RECURSOS ORDINARIOS


In [72]:
# O que são essas entradas duplicadas? Pq que existe elas?
num_entradas = len(pagamentos)
pagamentos = pagamentos.drop_duplicates()
num_entradas_unicas = len(pagamentos)
print(f'Numero de entradas duplicadas: {num_entradas - num_entradas_unicas}\nNumero de entradas unicas: {num_entradas_unicas}')

Numero de entradas duplicadas: 21316
Numero de entradas unicas: 160211


In [68]:
pagamentos.head()
# for i in range(10):
#     print(pagamentos.loc[i, 'bem_ou_servico_prestado'])

,data_publicacao,fase,credor,valor,numero,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
0,01/01/2010,EMPENHO,metalpires industria lt,203690.0,10000348,06.965.538/0001-00,01/01/2010,157/2009,ref aquisicao mobiliarios equipamentos diversa...,out equip mat permanente,equipar reequipar unidades escolares rede muni...,educacao,ensino fundamental,pregao,rec imp transf educacao
1,01/01/2010,EMPENHO,inst nac seguridade social,1500000.0,10000195,01.404.357/0001-51,01/01/2010,NaN,referente despesa parce lamento divida inss pr...,inss parcelamento,administracao encagos especiais,encargos especiais,servico divida interna,isento,recursos ordinarios
2,01/01/2010,EMPENHO,gasauto locac veiculos l,18816.0,113/2009,00.426.288/0001-58,01/01/2010,039/2009,locac motocicletas partida servir smt partir e...,locacao veiculos,seguranca transito,transporte,servicos urbanos,convite,rec proprias ent adm
3,02/01/2010,EMPENHO,salustiano jose alves dias,4800.0,10000024,407.981.865-34,02/01/2010,230/2007,termo renovacao aditivo referente loca cao cam...,locacao veiculos,pavimentacao infra estrutura,urbanismo,infra estrutura urbana,tomada de preco,recursos ordinarios
4,02/01/2010,EMPENHO,valdeane santos dias,3600.0,10000494,006.534.185-66,02/01/2010,NaN,ref aluguel imovel onde funciona nucleo progra...,locacao imoveis,programa errad trabalho infantil peti,assistencia social,assistencia crianca ado,isento,recursos ordinarios


# Investigar
Do print acima a gente pode ver que:

Das 427.873 entradas, só existem 193.494 números.

Existem 8831 credores (nome) mas só 7828 (cpnj). Por que? Typos etc? Conferir

Existem 193.494 números, mas só 17.919 números de processo. O que representam esses números | números de processo


In [316]:
# Investigando numero x numero_processo. O que significa numero?
gp_numero = pagamentos.groupby('numero')
print(gp_numero.count().sort_values('numero_processo', ascending=False))
print ('\n\n- # # # # # # -\n\n')
gp_numero.nunique().sort_values('numero_processo', ascending=False)

KeyError: 'numero'

In [317]:
pagamentos[pagamentos['numero'] == '440/14CP'].head()

KeyError: 'numero'

In [318]:
# Conferindo discrepancia entre credores e cpf_ou_cnpj (documento)
cpf_group = pagamentos.groupby('cpf_ou_cnpj').nunique().sort_values('credor', ascending=False)
cpf_group.head(10)

KeyError: 'cpf_ou_cnpj'

In [319]:
# Investigando cnpj com multiplos credores 
credor = pagamentos[pagamentos['cpf_ou_cnpj'] == '07.520.940/0001-50']
credor.groupby('credor').head()


KeyError: 'cpf_ou_cnpj'

## Número de CPFs/CNPJs

O CNPJ 14.043.574/0001-51 tem 169 credores inscritos sob ele. Este CNPJ é da Prefeitura.

O CNPJ 01.404.357/0001-51 tem 154 e não parece ser um CNPJ válido tb.

O CPF 000.000.000-00 tem 69 e não parece ser um CPF válido tb.

O CNPJ 08.576.590/0001-07 tem 23 inscritos. É do Fundo de Saúde Municipal de Feira

O CNPJ 00.857.659/0001-07 tem  19. Não achei origem.

O CNPJ 07.520.940/0001-50 tem 18. É da Fundação Egberto Tavares.


Difícil. Vale mais a pena agrupar pelo nome do credor do que pelo documento, que seria o certo.

## Pagamentos negativos

Valores negativos fazem sentido pra pagamentos?

Vamos tentar ter uma ideia melhor do que sejam estes pagamentos negativos:

In [1]:
negative_payments = raw_pagamentos[raw_pagamentos['valor'] < 0]
print(negative_payments.info())
negative_payments

NameError: name 'raw_pagamentos' is not defined

Munidos dos numeros de processo, vamos ao [Portal da Transparência](http://www.transparencia.feiradesantana.ba.gov.br/index.php?view=despesa) conferir o que são estes valores.

O primeiro, do Banco do Brasil, constam no mesmo dia 3 pagamentos: este negativo, outro de mesmo valor e positivo, outro de valor parecido e positivo. 

Será que quem registrou, registrou primeiro um valor errado, depois um valor igual e negativo (pra anular o errado) e depois um outro valor, dessa vez correto? Vamos olhar os outros pra ver se seguem este padrão

O segundo, do IUNE -INST. DE. URO E NEFRO FSA, existem 10 pagamentos similares (5 de cada valor). Ao final, no balanço da conta, resta apenas um pagamento de cada valor. Parece corroborar com a hipótese anterior.

O terceiro, de SIMOES ESTRELAS EMPREENDIMENTOS LTDA ME EPP, também segue o mesmo padrão. 5 pagamentos que, somados, correspondem a 1 pagamento de um valor.

Vou considerar a hipótese como correta após estas 3 análises pra partir pra análise dos dados, o que não é ideal. No entanto, se cada um que ler este notebook analisar +3 credores / CNPJs e fizer um commit aqui incluindo a análise destes, ao final teremos uma análise correta sobre o que aconteceu.

De acordo com esta hipótese, eu posso simplesmente descartar os valores negativos, pq foram usados apenas pra balancear entradas duplicadas ou erradas. Como as entradas duplicadas foram removidas (50.316 delas, inclusive), isto não constitui um problema. 

Seguiremos com as entradas erradas, no entanto, na fé que não alterem muito o resultado. No total, existem apenas 32 pagamentos negativos não repetidos no ano de 2018 (ano em análise). Dos 3 analisados, apenas o pagamento real (o balanço) do BB constituia um valor diferente, não repetido, do pagamento que foi cancelado. No final das contas teremos um pagamento duplicado do BB. Com sorte, foram poucos casos onde isso ocorreu.

